In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import mariadb
import sys

import os
from dotenv import load_dotenv

class DBConnection:
    @staticmethod
    def getConnection():
        load_dotenv(override=True)
        try:
            conn = mariadb.connect(
                user=os.environ['DATABASE_USERNAME'],
                password=os.environ['DATABASE_PASSWORD'],
                host=os.environ['HOST_CONNECT'],
                port=int(os.environ['DATABASE_PORT']),
                database=os.environ['DATABASE_NAME']

            )
        except mariadb.Error as e:
            print(f"Error connecting to MariaDB Platform: {e}")
            sys.exit(1)
            
        return conn.cursor(dictionary=True)

conn = DBConnection.getConnection()


ModuleNotFoundError: No module named 'numpy'

In [ ]:
conn.execute("""
            SELECT 
                wp_posts.ID,
                wp_posts.post_title,
                Round(Avg(DISTINCT(wp_commentmeta.meta_value)), 1) AS nps,
                COUNT(DISTINCT(wp_wc_order_product_lookup.product_id)) as qt_orders,
                COUNT(DISTINCT(wp_commentmeta.meta_id)) as rating_count,
                Round(wp_wc_product_meta_lookup.max_price, 2) as current_price,
                wp_wc_product_meta_lookup.stock_quantity,
                CASE 
                    WHEN wp_wc_product_meta_lookup.stock_status = "instock" THEN 1
                    ELSE 0
                END as stock_status
            FROM   wp_posts
                LEFT JOIN wp_comments
                        ON wp_posts.id = wp_comments.comment_post_id
                LEFT JOIN wp_commentmeta
                        ON wp_comments.comment_id = wp_commentmeta.comment_id
                LEFT JOIN wp_wc_order_product_lookup
                        ON wp_posts.ID = wp_wc_order_product_lookup.product_id
                LEFT JOIN wp_wc_product_meta_lookup
                        ON wp_posts.ID = wp_wc_product_meta_lookup.product_id
            WHERE 
            (wp_commentmeta.meta_key = 'rating' OR wp_commentmeta.meta_key is null)
            AND wp_posts.post_type = "product"
            GROUP BY wp_posts.post_title  
        """)

products_kpis_1 = pd.DataFrame(self.conn.fetchall())

products_kpis_list = []

for index, row in products_kpis_1.iterrows():
    index_recompra = self.get_index_recompra(row['ID'])
    new_row = dict({
        "id": row['ID'],
        "post_title": row['post_title'],
        "current_price": row['current_price'],
        "nps": row['nps'],
        "stock_quantity": row['stock_quantity'],
        "stock_status": row['stock_status'],
        "qt_orders": row['qt_orders'],
        "rating_count": row['rating_count'],
        "repurchase": index_recompra,
    })
    products_kpis_list.append(new_row)

products_kpi = pd.DataFrame(products_kpis_list)

products_kpi['nps'] = products_kpi['nps'].fillna(0)
products_kpi['stock_quantity'] = products_kpi['stock_quantity'].fillna(99999)

X = np.array(products_kpi.drop(['post_title', 'id', 'current_price'], axis=1))

kmeans = KMeans(n_clusters=3, random_state=0).fit(X)

classes = list(map(self.padronizar_classe, kmeans.labels_))
products_kpi['K_classes'] = classes

scaler = StandardScaler()

scaled_data = scaler.fit_transform(pedidos)

scaled_orders = pd.DataFrame(scaled_data)


products_kpi = products_kpi.sort_values(by='K_classes')